In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
from exp.nb_11 import *

## Serializing the model 

In [3]:
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)

In [5]:
??RandomResizedCrop

In [6]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name="val"))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [15]:
len(il)

13394

In [17]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [21]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [22]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [25]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [26]:
learn.fit(40, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.720706,0.480534,1.481868,0.620000,00:25
1,1.367667,0.642082,1.296710,0.668000,00:25
2,1.257709,0.692958,1.252022,0.678000,00:26
3,1.197153,0.713510,1.184464,0.744000,00:27
4,1.163692,0.728866,1.263689,0.724000,00:27
5,1.132486,0.742438,1.322580,0.644000,00:27
6,1.102522,0.756864,1.220877,0.724000,00:27
7,1.077420,0.766636,1.047767,0.780000,00:27
8,1.052630,0.779277,1.019637,0.806000,00:27
9,1.031094,0.789437,1.053998,0.772000,00:27


In [27]:
st = learn.model.state_dict()

In [28]:
type(st)

collections.OrderedDict

In [29]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [30]:
st['10.bias']

tensor([ 0.0142, -0.0115, -0.0174, -0.0236, -0.0480,  0.0233,  0.0116,  0.0084,
         0.0100,  0.0333], device='cuda:0')

In [31]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

In [32]:
torch.save(st, mdl_path/"iw5")

## Pets 

In [33]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [34]:
pets.ls()

[PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/annotations'),
 PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images')]

In [35]:
pets_path = pets/'images'

In [36]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [37]:
il

ImageList (7390 items)
[PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_27.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/British_Shorthair_90.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/english_cocker_spaniel_106.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/Birman_37.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/beagle_175.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_53.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/miniature_pinscher_131.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/Persian_263.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/american_bulldog_180.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/american_bulldog_23.jpg')...]
Path: /home/ricky/.fastai/data/oxford-iiit-pet/images

In [38]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [39]:
random.seed(42)

In [41]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [42]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_27.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/english_cocker_spaniel_106.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/Birman_37.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/beagle_175.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_53.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/miniature_pinscher_131.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/american_bulldog_180.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/beagle_85.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/keeshond_59.jpg'), PosixPath('/home/ricky/.fastai/data/oxford-iiit-pet/images/american_pit_bull_terrier_139.jpg')...]
Path: /home/ricky/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/home/ricky/.fastai

In [47]:
n = il.items[0].name; n

'wheaten_terrier_27.jpg'

In [48]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'wheaten_terrier'

In [54]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [55]:
proc = CategoryProcessor()

In [56]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [57]:
", ".join(proc.vocab)

'wheaten_terrier, english_cocker_spaniel, Birman, beagle, miniature_pinscher, american_bulldog, keeshond, american_pit_bull_terrier, pug, staffordshire_bull_terrier, Egyptian_Mau, Bombay, japanese_chin, english_setter, Bengal, Abyssinian, newfoundland, chihuahua, Sphynx, Siamese, boxer, leonberger, german_shorthaired, samoyed, Persian, British_Shorthair, Maine_Coon, Russian_Blue, great_pyrenees, pomeranian, havanese, yorkshire_terrier, shiba_inu, saint_bernard, basset_hound, Ragdoll, scottish_terrier'

In [58]:
ll.valid.x.tfms = val_tfms

In [59]:
c_out = len(proc.vocab)

In [61]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [64]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [65]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.468439,0.088796,3.446444,0.096819,00:15
1,3.288485,0.136943,4.663083,0.081604,00:15
2,3.060177,0.196040,3.060615,0.195021,00:15
3,2.760451,0.277186,2.542052,0.349931,00:15
4,2.455071,0.383531,2.387991,0.410788,00:15


## Custom head 

In [66]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [67]:
st = torch.load(mdl_path/'iw5')

In [68]:
m = learn.model

In [70]:
m.load_state_dict(st)

<All keys matched successfully>

In [73]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [80]:
m

XResNet(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (1): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): ResBlock(
      (convs): Sequential(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, mome

In [74]:
cut

8

In [76]:
xb, yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [77]:
pred = m_cut(xb)

In [78]:
pred.shape

torch.Size([128, 512, 4, 4])

In [81]:
ni = pred.shape[1]; ni

512

In [83]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [84]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))

In [85]:
learn.model = m_new

In [86]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.309101,0.149543,2.738587,0.297372,00:14
1,2.413152,0.387131,2.316097,0.408022,00:15
2,2.000330,0.533523,2.206379,0.452282,00:15
3,1.725286,0.623819,1.689017,0.637621,00:15
4,1.481266,0.726114,1.583364,0.674965,00:15


In [87]:
c_out

37

## adpat_model and gradual unfreezing 

In [108]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(m.children())
               if isinstance(o, nn.AdaptiveAvgPool2d))
    m_cut = m[:cut]
    xb, yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut, AdaptiveConcatPool2d(), Flatten(),
        nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [109]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

<All keys matched successfully>

In [110]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [111]:
# freeze body of xresnet
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [112]:
learn.model[3]

Linear(in_features=1024, out_features=37, bias=True)

In [113]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.129789,0.548523,1.787814,0.604426,00:07
1,1.626376,0.682016,1.790332,0.609959,00:07
2,1.451739,0.737963,1.624430,0.658368,00:07


In [114]:
# unfreeze
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [115]:
learn.fit(10, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.282897,0.805910,1.486606,0.712310,00:15
1,1.218161,0.831108,1.521279,0.704011,00:15
2,1.231035,0.822859,1.784861,0.614108,00:15
3,1.275853,0.793310,1.723762,0.637621,00:15
4,1.274065,0.800360,1.761314,0.637621,00:15
5,1.252421,0.809959,1.667100,0.656985,00:15
6,1.163459,0.849407,1.528123,0.702628,00:15
7,1.044003,0.900255,1.466336,0.734440,00:15
8,0.975678,0.925604,1.372496,0.762102,00:15
9,0.940579,0.938353,1.383621,0.763485,00:15


## Batch norm transfer 

In [116]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [117]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)

def set_grad(m, b):
    if isinstance(m, (nn.Linear,nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [118]:
apply_mod(learn.model, partial(set_grad, b=False))

In [119]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.745601,0.743513,1.536568,0.716459,00:11
1,1.120332,0.867707,1.563349,0.684647,00:11
2,1.018924,0.905655,1.401594,0.751037,00:11


In [120]:
apply_mod(learn.model, partial(set_grad, b=True))

In [121]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.975116,0.919454,1.455162,0.742739,00:14
1,1.069618,0.880456,1.838809,0.614108,00:14
2,1.127058,0.857957,1.769375,0.648686,00:14
3,1.083745,0.876256,1.472421,0.746888,00:14
4,0.940960,0.932803,1.382909,0.757953,00:14


In [122]:
#pytorch has its own apply
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and param groups 

In [123]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [124]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [125]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1,g2

In [126]:
a,b = bn_splitter(learn.model)

In [127]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [128]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [129]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [131]:
cb_types.after_backward

'after_backward'

In [132]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

In [133]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [134]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [135]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [136]:
def _print_det(o): 
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [139]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.408272,0.819259,2.199185,0.543568,00:11
1,1.328171,0.790610,1.788380,0.655602,00:11
2,1.144012,0.860807,1.449325,0.737206,00:11


In [140]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [141]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.024578,0.905205,1.515402,0.728907,00:11
1,1.046057,0.896805,1.509977,0.727524,00:11
2,1.023174,0.905205,1.448250,0.757953,00:11
3,0.984140,0.923204,1.432801,0.760719,00:11
4,0.941695,0.934603,1.414534,0.770401,00:11


## Export 

In [143]:
!python notebook2script.py 11a_transfer_learning.ipynb

Converted 11a_transfer_learning.ipynb to exp/nb_11a.py
